In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q transformers datasets sentencepiece
!pip install -q safetensors
!pip uninstall -y -q transformers accelerate
!pip install -q transformers accelerate
!pip install -q evaluate
# !pip install -q parsivar

!pip install -q git+https://github.com/huggingface/accelerate

!git clone -q https://github.com/huggingface/transformers.git
!cd transformers
!pip install  -q -e.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.4/302.4 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: file:///content does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.


In [ ]:
import json
import torch
from transformers import MBart50TokenizerFast, MBartForConditionalGeneration
from transformers import AutoTokenizer, MT5ForConditionalGeneration
from datasets import Dataset
from tqdm.auto import tqdm
import collections
from transformers import AutoModelForQuestionAnswering
from transformers import Trainer
import evaluate
import os
import numpy as np
from sklearn.preprocessing import PowerTransformer
import statistics

In [ ]:
max_length = 384
stride = 128

def preprocess_validation_examples(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["pq_id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    inputs["example_id"] = example_ids
    return inputs

# save only BART predictions

In [ ]:
metric = evaluate.load("squad")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_name_or_path = 'NegarSH/mbart_extratranslations2'
tokenizer = MBart50TokenizerFast.from_pretrained(model_name_or_path)
model = MBartForConditionalGeneration.from_pretrained(model_name_or_path, use_safetensors=True).to(device)

for each in os.listdir('/content/drive/MyDrive/my_quran_dataset'):
  models_results = []
  # model_name_or_path = 'NegarSH/mbart_extratranslations2'
  # tokenizer = MBart50TokenizerFast.from_pretrained(model_name_or_path)
  # model = MBartForConditionalGeneration.from_pretrained(model_name_or_path, use_safetensors=True).to(device)

  dev_path = '/content/drive/MyDrive/my_quran_dataset/' + each

  with open(dev_path, 'r', encoding='utf-8-sig') as f:
    dev_data_list = json.load(f)

  predicted_output = []
  ground_truth = []

  predicted_answers = []
  theoretical_answers = []

  for qa in dev_data_list:
    ground_truth.append(qa['answers'][0]['text'].strip())

    question = qa['question']
    context = qa['context']


    input_text =  f"answer: {qa['question']} context: {qa['context']} </s>"
    input_ids = tokenizer(input_text,
                          return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).input_ids.to(device)

    attention_mask = tokenizer(input_text,
                          return_tensors='pt', pad_to_max_length=True, truncation=True, max_length=64).attention_mask.to(device)

    generated_answer = model.generate(input_ids, attention_mask = attention_mask,
                                  max_length=64, output_scores = True, return_dict_in_generate = True)


    generated_answers = tokenizer.batch_decode(generated_answer["sequences"], skip_special_tokens=True)
    answer_scores = generated_answer["sequences_scores"]

    answers_dict = {}
    for ans, score in zip(generated_answers, answer_scores):
      if ans and ans != '[CLS]' and ans != '[SEP]' and ans in context and len(ans) <= 30 and len(ans)>0:
          answers_dict[ans] = score

    try:
      final_answer = max(answers_dict, key=answers_dict.get)
      final_score = answers_dict[final_answer]
    except:
      final_answer = ''
      final_score = 0
    predicted_answers.append({"id": qa['pq_id'], "prediction_text": final_answer.strip(), "score": final_score})
    predicted_output.append(final_answer.strip())

    new_ex = {}
    new_ex['id'] = qa["pq_id"]
    new_ex['answers'] = {}
    new_ex['answers']['answer_start'] = [qa['answers'][0]['start_char']]
    new_ex['answers']['text'] = [qa['answers'][0]['text']]
    theoretical_answers.append(new_ex)

  models_results.append(predicted_answers)
  translation = each.split('.')[0]
  sava_path = '/content/drive/MyDrive/models_predictions/only_bart/' + translation + '.pt'
  torch.save(models_results, sava_path)


# predictions for dev_v9

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np

def standardize(numbers):
  scaler = StandardScaler()
  data = np.array(numbers).reshape(-1, 1)
  scaler.fit(data)
  scaled_data = scaler.transform(data)

  # t5_normalized = normalize_list(scaled_data)
  t5_normalized = scaled_data.flatten().tolist()

  return t5_normalized

In [ ]:
def normalize_list2(numbers):
    min_value = min(numbers)
    max_value = max(numbers)

    normalized_numbers = [(x - min_value) / (max_value - min_value) for x in numbers]

    return normalized_numbers

In [ ]:

power_transformer = PowerTransformer()

metric = evaluate.load("squad")

em = {}
f1 = {}

for pred in os.listdir('/content/drive/MyDrive/models_predictions'):
  torch_path = '/content/drive/MyDrive/models_predictions/' + pred
  if os.path.isfile(torch_path) :
    bart_path = '/content/drive/MyDrive/models_predictions/only_bart/' + pred

    with open(torch_path, 'rb')as f:
      models_results = torch.load(f, map_location=torch.device('cpu'))

    with open(bart_path, 'rb')as f:
      bart_results = torch.load(f, map_location=torch.device('cpu'))

    mbart_scores = []
    for each in bart_results[0]:
      try:
        score = each['score'].item()
        mbart_scores.append(score)
      except:
        mbart_scores.append(0)


    albert_scores = []
    for each in models_results[1]:
      score = each['score'].item()
      albert_scores.append(score)

    roberta_scores = []
    for each in models_results[2]:
      score = each['score'].item()
      roberta_scores.append(score)

    mbert_scores = []
    for each in models_results[3]:
      score = each['score'].item()
      mbert_scores.append(score)

    parsbert_scores = []
    for each in models_results[4]:
      score = each['score'].item()
      parsbert_scores.append(score)

    # data = np.array(mbart_scores).reshape(-1, 1)
    # power_transformer.fit(data)
    # transformed_data = power_transformer.transform(data)
    # mbart_normalized = transformed_data.flatten().tolist()

    mbart_normalized = standardize(mbart_scores)
    # roberta_normalized = standardize(roberta_scores)
    # albert_normalized = standardize(albert_scores)
    # mbert_normalized = standardize(mbert_scores)
    # parsbert_normalized = standardize(parsbert_scores)

    roberta_normalized = normalize_list2(roberta_scores)
    albert_normalized = normalize_list2(albert_scores)
    mbert_normalized = normalize_list2(mbert_scores)
    parsbert_normalized = normalize_list2(parsbert_scores)

    models_results2  = [bart_results[0], models_results[1] , models_results[2], models_results[3], models_results[4]]

    for i in range(len(models_results[0])):
      models_results2[0][i]['score'] = mbart_normalized[i]

    for i in range(len(models_results[1])):
      models_results2[1][i]['score'] = albert_normalized[i]

    for i in range(len(models_results[2])):
      models_results2[2][i]['score'] = roberta_normalized[i]

    for i in range(len(models_results[3])):
      models_results2[3][i]['score'] = mbert_normalized[i]

    for i in range(len(models_results[4])):
      models_results2[4][i]['score'] = parsbert_normalized[i]

    dev_path = '/content/drive/MyDrive/my_quran_dataset/' + pred.split('.')[0] + '.json'

    with open(dev_path, 'r', encoding = 'utf-8-sig') as f:
      dev_data_list = json.load(f)

    dev_data = Dataset.from_list(dev_data_list)

    examples = dev_data

    theoretical_answers = []
    for ex in examples:
      new_ex = {}
      new_ex['id'] = ex["pq_id"]
      new_ex['answers'] = {}
      new_ex['answers']['answer_start'] = [ex['answers'][0]['start_char']]
      new_ex['answers']['text'] = [ex['answers'][0]['text']]
      theoretical_answers.append(new_ex)

    predicted_outputs = []

    for i in range(len(examples)):
      maximum = 0
      answer = ''
      for result in [models_results2[2], models_results2[0]]:
        if result[i]['score'] > maximum:
          maximum = result[i]['score']
          answer = result[i]['prediction_text']

      predicted_outputs.append({"id": examples[i]['pq_id'], "prediction_text": answer})

    metric_dict = metric.compute(predictions=predicted_outputs, references=theoretical_answers)

    em[pred.split('.')[0]] = metric_dict['exact_match']
    f1[pred.split('.')[0]] = metric_dict['f1']


In [ ]:
statistics.mean(f1.values())

90.7941715662136

In [ ]:
statistics.mean(em.values())

87.40673062281103

In [ ]:
power_transformer = PowerTransformer()
metric = evaluate.load("squad")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

with open('/content/drive/MyDrive/original_data_predictions.pt', 'rb')as f:
  models_results = torch.load(f, map_location=torch.device('cpu'))

mbart_scores = []
for each in models_results[0]:
  try:
    score = each['score'].item()
    mbart_scores.append(score)
  except:
    mbart_scores.append(0)


albert_scores = []
for each in models_results[1]:
  score = each['score'].item()
  albert_scores.append(score)

roberta_scores = []
for each in models_results[2]:
  score = each['score'].item()
  roberta_scores.append(score)

mbert_scores = []
for each in models_results[3]:
  score = each['score'].item()
  mbert_scores.append(score)

parsbert_scores = []
for each in models_results[4]:
  score = each['score'].item()
  parsbert_scores.append(score)

# mt5_scores = []
# for each in models_results[5]:
#   score = each['score'].item()
#   mt5_scores.append(score)

data = np.array(mbart_scores).reshape(-1, 1)
power_transformer.fit(data)
transformed_data = power_transformer.transform(data)
mbart_normalized = transformed_data.flatten().tolist()

# data2 = np.array(mt5_scores).reshape(-1, 1)
# power_transformer.fit(data2)
# transformed_data2 = power_transformer.transform(data2)
# mt5_normalized = transformed_data2.flatten().tolist()

# mbart_normalized = standardize(mbart_scores)

# roberta_normalized = standardize(roberta_scores)
# albert_normalized = standardize(albert_scores)
# mbert_normalized = standardize(mbert_scores)
# parsbert_normalized = standardize(parsbert_scores)

roberta_normalized = normalize_list2(roberta_scores)
albert_normalized = normalize_list2(albert_scores)
mbert_normalized = normalize_list2(mbert_scores)
parsbert_normalized = normalize_list2(parsbert_scores)

models_results2  = [models_results[0], models_results[1] , models_results[2], models_results[3], models_results[4], models_results[5]]

for i in range(len(models_results[0])):
  models_results2[0][i]['score'] = mbart_normalized[i]

for i in range(len(models_results[1])):
  models_results2[1][i]['score'] = albert_normalized[i]

for i in range(len(models_results[2])):
  models_results2[2][i]['score'] = roberta_normalized[i]

for i in range(len(models_results[3])):
  models_results2[3][i]['score'] = mbert_normalized[i]

for i in range(len(models_results[4])):
  models_results2[4][i]['score'] = parsbert_normalized[i]

dev_path = '/content/drive/MyDrive/dev_10_percent2.json'

with open(dev_path, 'r', encoding = 'utf-8-sig') as f:
  dev_data_list = json.load(f)

dev_data = Dataset.from_list(dev_data_list)

examples = dev_data

theoretical_answers = []
for ex in examples:
  new_ex = {}
  new_ex['id'] = ex["pq_id"]
  new_ex['answers'] = {}
  new_ex['answers']['answer_start'] = [ex['answers'][0]['start_char']]
  new_ex['answers']['text'] = [ex['answers'][0]['text']]
  theoretical_answers.append(new_ex)

predicted_outputs = []

for i in range(len(examples)):
  maximum = 0
  answer = ''
  # for result in [models_results2[3]]:
  for result in [models_results2[2], models_results2[3], models_results2[0]]:
    if result[i]['score'] > maximum:
      maximum = result[i]['score']
      answer = result[i]['prediction_text']

  predicted_outputs.append({"id": examples[i]['pq_id'], "prediction_text": answer})

metric.compute(predictions=predicted_outputs, references=theoretical_answers)

{'exact_match': 84.27260812581913, 'f1': 86.70592394005327}

In [ ]:
models_results2[0]

[{'id': 's2.29', 'prediction_text': 'زمین', 'score': 0.22925747955548662},
 {'id': 's12.43', 'prediction_text': 'پادشاه', 'score': 0.22888142829196817},
 {'id': 's16.96', 'prediction_text': '', 'score': 0.22926601814006797},
 {'id': 's57.14', 'prediction_text': 'صدا', 'score': -1.9602523758430836},
 {'id': 's86.11',
  'prediction_text': 'آسمان پرباران',
  'score': -0.019468193022132603},
 {'id': 's17.23', 'prediction_text': 'پیری', 'score': -0.017295218344536123},
 {'id': 's15.26', 'prediction_text': '', 'score': 0.22926601814006797},
 {'id': 's60.13', 'prediction_text': '', 'score': 0.22926601814006797},
 {'id': 's5.114', 'prediction_text': 'مائده', 'score': -11.298981462764015},
 {'id': 's14.49', 'prediction_text': 'مجرمان', 'score': 0.22924636977507049},
 {'id': 's17.78', 'prediction_text': 'قرآن فجر', 'score': 0.22408110605316467},
 {'id': 's12.41',
  'prediction_text': 'ساقی شراب',
  'score': 0.22918265168539448},
 {'id': 's76.18', 'prediction_text': 'سلسبیل', 'score': 0.229236793